<a href="https://colab.research.google.com/github/niklashaffert/LandUseClassification/blob/main/01_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook is used to download the data for the project from kaggle, then create a pandas data frame for each of the training, validation and test data and save it in drive.

# Settings

## Libraries

In [1]:
from google.colab import drive
from kagglehub import dataset_download
from numpy import array
from os import path
from pandas import read_csv, DataFrame
from PIL import Image

Load custom functions from GitHub.

In [8]:
!rm functions.py
!wget https://raw.githubusercontent.com/niklashaffert/LandUseClassification/main/functions.py
from functions import store_drive

--2025-02-23 19:36:16--  https://raw.githubusercontent.com/niklashaffert/LandUseClassification/main/functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228 [text/plain]
Saving to: ‘functions.py’

functions.py        100%[===================>]     228  --.-KB/s    in 0s      

2025-02-23 19:36:16 (3.91 MB/s) - ‘functions.py’ saved [228/228]



## Drive Settings

In [3]:
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/LandUseClassification'

Mounted at /content/drive


# Load Data from kaggle

In [4]:
path_datasets = dataset_download('apollo2506/landuse-scene-classification')

100%|██████████| 1.98G/1.98G [00:30<00:00, 70.1MB/s]

Extracting files...


# Generate Data Frames with Images

In [5]:
@staticmethod
def load_data(name_csv):
    data_csv = read_csv(path.join(path_datasets, name_csv))
    # Create a list to store the data for the DataFrame
    data = []
    success_count = 0
    failure_count = 0

    # Set wanted size of output image
    fixed_size = (224, 224)  # Match ResNet-18 input size

    # Iterate through the data
    for i in range(len(data_csv)):
        image_path = path.join(path_datasets, 'images/', data_csv.iloc[i, 1])
        try:
            img = Image.open(image_path)
            img = img.resize(fixed_size)
            data.append({
                'Image': img,
                'Filename': data_csv.iloc[i, 1],
                'Label': data_csv.iloc[i, 2],
                'ClassName': data_csv.iloc[i, 3],
                'ImageArray': array(img).flatten()
            })
            success_count += 1
        except Exception as e:
            failure_count += 1
            print(f"Failed to load image {image_path}: {e}")

    print(f"Successfully loaded {success_count} images into DataFrame.")
    print(f"Failed to load {failure_count} images.")

    # Create the DataFrame
    df = DataFrame(data)
    return df

In [6]:
df_train = load_data('train.csv')
df_validation = load_data('validation.csv')
df_test = load_data('test.csv')

Successfully loaded 7350 images into DataFrame.
Failed to load 0 images.
Successfully loaded 2100 images into DataFrame.
Failed to load 0 images.
Successfully loaded 1050 images into DataFrame.
Failed to load 0 images.


# Store Data in Drive

In [9]:
store_drive(df_train, path.join(drive_path, 'Data/df_train.pkl'))
store_drive(df_validation, path.join(drive_path, 'Data/df_validation.pkl'))
store_drive(df_test, path.join(drive_path, 'Data/df_test.pkl'))